In [230]:
# Dependencies
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import normalize
from matplotlib import pyplot as plt

In [3]:
# Load in data sets
teams = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/mteams.csv")
tourney_results = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyCompactResults.csv")
tourney_results_detailed = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv")
seeds = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySeeds.csv")
regular_season = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonCompactResults.csv")
regular_season_detailed = pd.read_csv("ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv")

In [4]:
# Inspect data
regular_season

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
166875,2020,128,1204,81,1209,62,A,0
166876,2020,128,1402,85,1111,68,H,0
166877,2020,128,1299,86,1224,77,N,0
166878,2020,128,1393,81,1314,53,N,0


In [5]:
# Format seeds to be a number only
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2281,2019,12,1332
2282,2019,13,1414
2283,2019,14,1330
2284,2019,15,1159


In [6]:
# Aggregate by season for wins and points
wins = regular_season.groupby(['Season', 'WTeamID'])['WTeamID'].count()
loses = regular_season.groupby(['Season', 'LTeamID'])['LTeamID'].count()
points_w = regular_season.groupby(['Season', 'WTeamID'])['WScore'].mean()
points_l = regular_season.groupby(['Season', 'LTeamID'])['LScore'].mean()

wins.index = wins.index.rename(['Season', 'TeamID'])
loses.index = loses.index.rename(['Season', 'TeamID'])
points_w.index = points_w.index.rename(['Season', 'TeamID'])
points_l.index = points_l.index.rename(['Season', 'TeamID'])

In [8]:
# Get records, PPG for each team each season
records = pd.merge(wins.reset_index(), loses.reset_index())
records = records.rename(columns={'WTeamID': 'Wins', 'LTeamID': 'Loses'})
records['win_percentage'] = records['Wins'] / (records['Wins'] + records['Loses'])
records = records.merge(points_w.reset_index()).merge(points_l.reset_index())
total_games = records['Wins'] + records['Loses']
records['ppg'] = (records['Wins'] / total_games) * records['WScore'] + (records['Loses'] / total_games) * records['LScore']
records = records.drop(columns = ['WScore', 'LScore'])
records

,Season,TeamID,Wins,Loses,win_percentage,ppg
0,1985,1102,5,19,0.208333,63.083333
1,1985,1103,9,14,0.391304,61.043478
2,1985,1104,21,9,0.700000,68.500000
3,1985,1106,10,14,0.416667,71.625000
4,1985,1108,19,6,0.760000,83.000000
...,...,...,...,...,...,...
11572,2020,1463,21,7,0.750000,74.535714
11573,2020,1464,16,15,0.516129,71.193548
11574,2020,1465,17,10,0.629630,76.222222
11575,2020,1466,11,17,0.392857,67.464286


In [10]:
# Combine with tournament data
combined_df = pd.merge(tourney_results, records, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
combined_df = combined_df.rename(columns={'Wins': 'WTeamSeasonWins', 'Loses': 'WTeamSeasonLoses', 
                                    'win_percentage': 'WTeamWinPercentage', 'ppg': 'WTeamPPG'})
combined_df = pd.merge(combined_df, records, left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
combined_df = combined_df.rename(columns={'Wins': 'LTeamSeasonWins', 'Loses': 'LTeamSeasonLoses', 
                                    'win_percentage': 'LTeamWinPercentage', 'ppg': 'LTeamPPG'})
combined_df = combined_df.drop(columns = ['TeamID_x', 'TeamID_y'])
combined_df = pd.merge(combined_df, seeds, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
combined_df = combined_df.drop(columns = ['TeamID'])
combined_df = combined_df.rename(columns={'Seed': 'WSeed'})
combined_df = pd.merge(combined_df, seeds, left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
combined_df = combined_df.drop(columns = ['TeamID'])
combined_df = combined_df.rename(columns={'Seed': 'LSeed'})
combined_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeasonWins,WTeamSeasonLoses,WTeamWinPercentage,WTeamPPG,LTeamSeasonWins,LTeamSeasonLoses,LTeamWinPercentage,LTeamPPG,WSeed,LSeed
0,1985,136,1116,63,1234,54,N,0,21,12,0.636364,65.333333,20,10,0.666667,69.733333,9,8
1,1985,136,1120,59,1345,58,N,0,18,11,0.620690,70.344828,17,8,0.680000,69.120000,11,6
2,1985,138,1120,66,1242,64,N,0,18,11,0.620690,70.344828,23,7,0.766667,76.033333,11,3
3,1985,136,1207,68,1250,43,N,0,25,2,0.925926,75.740741,11,18,0.379310,65.758621,1,16
4,1985,138,1207,63,1396,46,N,0,25,2,0.925926,75.740741,24,5,0.827586,65.620690,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2234,2019,152,1438,63,1120,62,N,0,29,3,0.906250,71.843750,25,9,0.735294,78.882353,1,5
2235,2019,154,1438,85,1403,77,N,1,29,3,0.906250,71.843750,26,6,0.812500,73.093750,1,3
2236,2019,137,1439,66,1387,52,N,0,24,8,0.750000,74.000000,23,12,0.657143,67.057143,4,13
2237,2019,139,1439,67,1251,58,N,0,24,8,0.750000,74.000000,25,6,0.806452,72.193548,4,12


In [11]:
# Convert win / loss to higher and lower seed, since game outcome is not determined until after playtime
upsets = combined_df[combined_df['WSeed'] > combined_df['LSeed']]
upsets = upsets.rename(columns = {'LTeamID': 'HighSeedTeamID', 'WTeamID': 'LowSeedTeamID',
                                       'LTeamSeasonWins': 'HighSeedSeasonWins', 'WTeamSeasonWins': 'LowSeedSeasonWins',
                                       'LTeamSeasonLoses': 'HighSeedSeasonLoses', 'WTeamSeasonLoses': 'LowSeedSeasonLoses',
                                       'LTeamWinPercentage': 'HighSeedWinPercentage', 'WTeamWinPercentage': 'LowSeedWinPercentage',
                                       'LTeamPPG': 'HighSeedPPG', 'WTeamPPG': 'LowSeedPPG',
                                       'LSeed': 'HighSeed', 'WSeed': 'LowSeed'})
upsets = upsets.drop(columns = ['WScore', 'LScore', 'WLoc', 'NumOT'])
upsets['HighSeedWon'] = 0
upsets

,Season,DayNum,LowSeedTeamID,HighSeedTeamID,LowSeedSeasonWins,LowSeedSeasonLoses,LowSeedWinPercentage,LowSeedPPG,HighSeedSeasonWins,HighSeedSeasonLoses,HighSeedWinPercentage,HighSeedPPG,LowSeed,HighSeed,HighSeedWon
0,1985,136,1116,1234,21,12,0.636364,65.333333,20,10,0.666667,69.733333,9,8,0
1,1985,136,1120,1345,18,11,0.620690,70.344828,17,8,0.680000,69.120000,11,6,0
2,1985,138,1120,1242,18,11,0.620690,70.344828,23,7,0.766667,76.033333,11,3,0
8,1985,136,1229,1425,20,7,0.740741,71.592593,19,9,0.678571,68.392857,9,8,0
10,1985,136,1246,1449,16,12,0.571429,65.607143,22,9,0.709677,65.129032,12,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,2019,145,1403,1211,26,6,0.812500,73.093750,30,3,0.909091,88.848485,3,1,0
2227,2019,152,1403,1277,26,6,0.812500,73.093750,28,6,0.823529,78.823529,3,2,0
2228,2019,137,1414,1243,29,5,0.852941,72.470588,25,8,0.757576,65.818182,13,4,0
2229,2019,137,1416,1433,23,8,0.741935,72.129032,25,7,0.781250,71.437500,9,8,0


In [12]:
# Same as above cell
favorites = combined_df[combined_df['LSeed'] >= combined_df['WSeed']]
favorites = favorites.rename(columns = {'WTeamID': 'HighSeedTeamID', 'LTeamID': 'LowSeedTeamID',
                                       'WTeamSeasonWins': 'HighSeedSeasonWins', 'LTeamSeasonWins': 'LowSeedSeasonWins',
                                       'WTeamSeasonLoses': 'HighSeedSeasonLoses', 'LTeamSeasonLoses': 'LowSeedSeasonLoses',
                                       'WTeamWinPercentage': 'HighSeedWinPercentage', 'LTeamWinPercentage': 'LowSeedWinPercentage',
                                       'WTeamPPG': 'HighSeedPPG', 'LTeamPPG': 'LowSeedPPG',
                                       'WSeed': 'HighSeed', 'LSeed': 'LowSeed'})
favorites = favorites.drop(columns = ['WScore', 'LScore', 'WLoc', 'NumOT'])
favorites['HighSeedWon'] = 1
favorites

,Season,DayNum,HighSeedTeamID,LowSeedTeamID,HighSeedSeasonWins,HighSeedSeasonLoses,HighSeedWinPercentage,HighSeedPPG,LowSeedSeasonWins,LowSeedSeasonLoses,LowSeedWinPercentage,LowSeedPPG,HighSeed,LowSeed,HighSeedWon
3,1985,136,1207,1250,25,2,0.925926,75.740741,11,18,0.379310,65.758621,1,16,1
4,1985,138,1207,1396,25,2,0.925926,75.740741,24,5,0.827586,65.620690,1,8,1
5,1985,143,1207,1260,25,2,0.925926,75.740741,25,5,0.833333,85.833333,1,4,1
6,1985,145,1207,1210,25,2,0.925926,75.740741,24,7,0.774194,70.903226,1,2,1
7,1985,152,1207,1385,25,2,0.925926,75.740741,27,3,0.900000,76.133333,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,2019,145,1438,1345,29,3,0.906250,71.843750,23,9,0.718750,76.187500,1,3,1
2234,2019,152,1438,1120,29,3,0.906250,71.843750,25,9,0.735294,78.882353,1,5,1
2235,2019,154,1438,1403,29,3,0.906250,71.843750,26,6,0.812500,73.093750,1,3,1
2236,2019,137,1439,1387,24,8,0.750000,74.000000,23,12,0.657143,67.057143,4,13,1


In [13]:
# Combine for training data
train_historic = pd.concat([favorites, upsets])

In [270]:
# Break DF into X and Y, and check shape to determine dimensions for neural network
X = train_historic[train_historic.columns[:-1]]
Y = train_historic[train_historic.columns[-1]]
X.shape

(2239, 14)

In [274]:
# use first 25 years for training, and last 10 for testing for about a 70/30 split
training_data = train_historic[train_historic['Season'] < 2010]
testing_data = train_historic[train_historic['Season'] >= 2010]

X_train = training_data[training_data.columns[:-1]]
Y_train = training_data[training_data.columns[-1]]
X_test = testing_data[testing_data.columns[:-1]]
Y_test = testing_data[testing_data.columns[-1]]

In [271]:
# Defining model with help from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# Use batch normalization layer for normalizing data, and dropout layers to reduce model bias
def define_model(input_df):
    model = Sequential();
    model.add(BatchNormalization(epsilon=0.001))
    model.add(Dense(256, input_dim = input_df.shape[1], activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [251]:
nn = define_model(X_train)

In [252]:
history = nn.fit(X_train, Y_train, epochs = 256, batch_size = 32)

Epoch 1/128
50/50 [==============================] - 1s 2ms/step - loss: 0.6152 - accuracy: 0.6733
Epoch 2/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5267 - accuracy: 0.7462
Epoch 3/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5362 - accuracy: 0.7282
Epoch 4/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5293 - accuracy: 0.7292
Epoch 5/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5364 - accuracy: 0.7166
Epoch 6/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5026 - accuracy: 0.7485
Epoch 7/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5080 - accuracy: 0.7417
Epoch 8/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5191 - accuracy: 0.7178
Epoch 9/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5139 - accuracy: 0.7397
Epoch 10/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5065 - accuracy: 0.7368
Epoch 11/

50/50 [==============================] - 0s 2ms/step - loss: 0.2508 - accuracy: 0.8829
Epoch 84/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2291 - accuracy: 0.9072
Epoch 85/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2142 - accuracy: 0.9210
Epoch 86/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2305 - accuracy: 0.9031
Epoch 87/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.9069
Epoch 88/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2083 - accuracy: 0.9094
Epoch 89/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2613 - accuracy: 0.8829
Epoch 90/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2252 - accuracy: 0.9073
Epoch 91/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2116 - accuracy: 0.9046
Epoch 92/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2336 - accuracy: 0.9017
Epoch 93/128


In [310]:
def plot_model_accuracy_and_loss(history, accuracy_name, loss_name):
    plt.plot(history.history['accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy Over Epochs')
    plt.savefig(accuracy_name, dpi=300, bbox_inches='tight')
    plt.clf()
    
    plt.plot(history.history['loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model Loss Over Epochs')
    plt.savefig(loss_name, dpi=300, bbox_inches='tight')
    plt.clf()
    
    
plot_model_accuracy_and_loss(history, 'model_accuracy_v1.png', 'model_loss_v1.png')

<Figure size 432x288 with 0 Axes>

In [249]:
# Try with 512 epochs
nn2 = define_model(X_train)
history512 = nn.fit(X_train, Y_train, epochs = 512, batch_size = 32)

Epoch 1/512
50/50 [==============================] - 0s 2ms/step - loss: 0.9245 - accuracy: 0.8037
Epoch 2/512
50/50 [==============================] - 0s 2ms/step - loss: 0.3462 - accuracy: 0.8467
Epoch 3/512
50/50 [==============================] - 0s 2ms/step - loss: 0.2858 - accuracy: 0.8866
Epoch 4/512
50/50 [==============================] - 0s 2ms/step - loss: 0.2718 - accuracy: 0.8828
Epoch 5/512
50/50 [==============================] - 0s 2ms/step - loss: 0.2215 - accuracy: 0.9126
Epoch 6/512
50/50 [==============================] - 0s 2ms/step - loss: 0.2173 - accuracy: 0.9107
Epoch 7/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1927 - accuracy: 0.9208
Epoch 8/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1910 - accuracy: 0.9259
Epoch 9/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1659 - accuracy: 0.9367
Epoch 10/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1870 - accuracy: 0.9329
Epoch 11/

50/50 [==============================] - 0s 2ms/step - loss: 0.1031 - accuracy: 0.9595
Epoch 84/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0946 - accuracy: 0.9576
Epoch 85/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0811 - accuracy: 0.9696
Epoch 86/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1158 - accuracy: 0.9544
Epoch 87/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1180 - accuracy: 0.9582
Epoch 88/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0956 - accuracy: 0.9677
Epoch 89/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0943 - accuracy: 0.9677
Epoch 90/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1128 - accuracy: 0.9595
Epoch 91/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1058 - accuracy: 0.9601
Epoch 92/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1067 - accuracy: 0.9633
Epoch 93/512


50/50 [==============================] - 0s 2ms/step - loss: 0.1120 - accuracy: 0.9582
Epoch 165/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0966 - accuracy: 0.9645
Epoch 166/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9671
Epoch 167/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0920 - accuracy: 0.9614
Epoch 168/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.9652
Epoch 169/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0870 - accuracy: 0.9671
Epoch 170/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0941 - accuracy: 0.9664
Epoch 171/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1096 - accuracy: 0.9652
Epoch 172/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0844 - accuracy: 0.9664
Epoch 173/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0908 - accuracy: 0.9639
Epoc

50/50 [==============================] - 0s 2ms/step - loss: 0.0850 - accuracy: 0.9683
Epoch 246/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1064 - accuracy: 0.9614
Epoch 247/512
50/50 [==============================] - 0s 2ms/step - loss: 0.1042 - accuracy: 0.9607
Epoch 248/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0750 - accuracy: 0.9728
Epoch 249/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0845 - accuracy: 0.9702
Epoch 250/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0988 - accuracy: 0.9652
Epoch 251/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0887 - accuracy: 0.9664
Epoch 252/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 0.9582
Epoch 253/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0889 - accuracy: 0.9671
Epoch 254/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0860 - accuracy: 0.9690
Epoc

50/50 [==============================] - 0s 2ms/step - loss: 0.0822 - accuracy: 0.9690
Epoch 327/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0861 - accuracy: 0.9709
Epoch 328/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0970 - accuracy: 0.9690
Epoch 329/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0844 - accuracy: 0.9715
Epoch 330/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0679 - accuracy: 0.9785
Epoch 331/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0796 - accuracy: 0.9721
Epoch 332/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0671 - accuracy: 0.9740
Epoch 333/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0561 - accuracy: 0.9823
Epoch 334/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0660 - accuracy: 0.9740
Epoch 335/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0587 - accuracy: 0.9785
Epoc

50/50 [==============================] - 0s 5ms/step - loss: 0.0697 - accuracy: 0.9740
Epoch 408/512
50/50 [==============================] - 0s 5ms/step - loss: 0.0833 - accuracy: 0.9747
Epoch 409/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0933 - accuracy: 0.9721
Epoch 410/512
50/50 [==============================] - 0s 4ms/step - loss: 0.0752 - accuracy: 0.9740
Epoch 411/512
50/50 [==============================] - 0s 4ms/step - loss: 0.0768 - accuracy: 0.9715
Epoch 412/512
50/50 [==============================] - 0s 4ms/step - loss: 0.0814 - accuracy: 0.9671
Epoch 413/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0778 - accuracy: 0.9734
Epoch 414/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0663 - accuracy: 0.9778
Epoch 415/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0855 - accuracy: 0.9696
Epoch 416/512
50/50 [==============================] - 0s 5ms/step - loss: 0.0809 - accuracy: 0.9709
Epoc

50/50 [==============================] - 0s 3ms/step - loss: 0.0858 - accuracy: 0.9671
Epoch 489/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0592 - accuracy: 0.9797
Epoch 490/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0616 - accuracy: 0.9797
Epoch 491/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0561 - accuracy: 0.9778
Epoch 492/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0712 - accuracy: 0.9810
Epoch 493/512
50/50 [==============================] - 0s 2ms/step - loss: 0.0725 - accuracy: 0.9690
Epoch 494/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0816 - accuracy: 0.9721
Epoch 495/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0723 - accuracy: 0.9740
Epoch 496/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0775 - accuracy: 0.9721
Epoch 497/512
50/50 [==============================] - 0s 3ms/step - loss: 0.0797 - accuracy: 0.9709
Epoc

In [254]:
# Try with 128 epochs
nn3 = define_model(X_train)
history128 = nn3.fit(X_train, Y_train, epochs = 128, batch_size = 32)

Epoch 1/128
50/50 [==============================] - 1s 2ms/step - loss: 0.6044 - accuracy: 0.7296
Epoch 2/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5300 - accuracy: 0.7354
Epoch 3/128
50/50 [==============================] - 0s 3ms/step - loss: 0.5442 - accuracy: 0.7158
Epoch 4/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5193 - accuracy: 0.7342
Epoch 5/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5216 - accuracy: 0.7318
Epoch 6/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5202 - accuracy: 0.7243
Epoch 7/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5244 - accuracy: 0.7155
Epoch 8/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5109 - accuracy: 0.7253
Epoch 9/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5052 - accuracy: 0.7258
Epoch 10/128
50/50 [==============================] - 0s 2ms/step - loss: 0.5118 - accuracy: 0.7289
Epoch 11/

50/50 [==============================] - 0s 2ms/step - loss: 0.2484 - accuracy: 0.8967
Epoch 84/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2377 - accuracy: 0.8943
Epoch 85/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2657 - accuracy: 0.8851
Epoch 86/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2486 - accuracy: 0.8846
Epoch 87/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2486 - accuracy: 0.8885
Epoch 88/128
50/50 [==============================] - 0s 3ms/step - loss: 0.2585 - accuracy: 0.8967
Epoch 89/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2265 - accuracy: 0.9094
Epoch 90/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2487 - accuracy: 0.8975
Epoch 91/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2554 - accuracy: 0.8842
Epoch 92/128
50/50 [==============================] - 0s 2ms/step - loss: 0.2153 - accuracy: 0.9065
Epoch 93/128


In [311]:
plot_model_accuracy_and_loss(history512, 'model_accuracy_v1_512.png', 'model_loss_v1_512.png')

<Figure size 432x288 with 0 Axes>

In [312]:
plot_model_accuracy_and_loss(history128, 'model_accuracy_v1_128.png', 'model_loss_v1_128.png')

<Figure size 432x288 with 0 Axes>

In [293]:
# Will use 256 epochs going forward. 
predictions = (nn.predict(X_test, batch_size = 32) > 0.5).astype("int32")
mean_squared_error(Y_test, predictions)

0.3212121212121212

In [294]:
accuracy_score(Y_test, predictions)

0.6787878787878788

In [295]:
confusion_matrix(Y_test, predictions)

array([[ 48, 136],
       [ 76, 400]], dtype=int64)

In [296]:
accuracy = nn.evaluate(X_train, Y_train)
print('Accuracy: ' + str(accuracy[1]))

50/50 [==============================] - 0s 2ms/step - loss: 0.0164 - accuracy: 0.9981
Accuracy: 0.9981000423431396


In [79]:
# Aggregate for detailed statistics; quite ugly but gets job done
wins = regular_season_detailed.groupby(['Season', 'WTeamID'])['WTeamID'].count()
loses = regular_season_detailed.groupby(['Season', 'LTeamID'])['LTeamID'].count()
points_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WScore'].mean()
points_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LScore'].mean()
fgm_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WFGM'].mean()
fgm_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LFGM'].mean()
fga_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WFGA'].mean()
fga_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LFGA'].mean()
fgm3_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WFGA3'].mean()
fgm3_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LFGA3'].mean()
ftm_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WFTM'].mean()
ftm_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LFTM'].mean()
fta_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WFTA'].mean()
fta_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LFTA'].mean()
or_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WOR'].mean()
or_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LOR'].mean()
dr_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WDR'].mean()
dr_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LDR'].mean()
ast_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WAst'].mean()
ast_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LAst'].mean()
to_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WTO'].mean()
to_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LTO'].mean()
stl_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WStl'].mean()
stl_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LStl'].mean()
blk_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WBlk'].mean()
blk_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LBlk'].mean()
pf_w = regular_season_detailed.groupby(['Season', 'WTeamID'])['WPF'].mean()
pf_l = regular_season_detailed.groupby(['Season', 'LTeamID'])['LPF'].mean()

dataframes = [wins, loses, points_w, points_l, fgm_w, fgm_l, fga_w, fga_l, fgm3_w, fgm3_l, ftm_w, ftm_l,
             fta_w, fta_l, or_w, or_l, dr_w, dr_l, ast_w, ast_l, to_w, to_l, stl_w, stl_l, blk_w, blk_l, 
             pf_w, pf_l]

for df in dataframes:
    df.index = df.index.rename(['Season', 'TeamID'])

In [96]:
records_v2 = wins.to_frame().reset_index()
for df in dataframes:
    records_v2 = records_v2.merge(df.to_frame().reset_index())

records_v2 = records_v2.rename(columns={'WTeamID': 'Wins', 'LTeamID': 'Loses'})

total_games = records_v2['Wins'] + records_v2['Loses']

column_pairs = [('WScore', 'LScore'), ('WFGM', 'LFGM'), ('WFGA', 'LFGA'), ('WFGA3', 'LFGA3'), ('WFTM', 'LFTM'),
               ('WFTA', 'LFTA'), ('WOR', 'LOR'), ('WDR', 'LDR'), ('WAst', 'LAst'), ('WTO', 'LTO'), ('WStl', 'LStl'),
               ('WBlk', 'LBlk'), ('WPF', 'LPF')]
new_col_name = ['ppg', 'fgm', 'fga', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']


for i, col in enumerate(column_pairs):
    records_v2[new_col_name[i]] = (records_v2['Wins'] / total_games) * records_v2[column_pairs[i][0]] + (records_v2['Loses'] / total_games) * records_v2[column_pairs[i][1]]
    records_v2 = records_v2.drop(columns = list(column_pairs[i]))

In [104]:
# Follows same logic as first dataset at this point
combined_df_v2 = pd.merge(tourney_results, records_v2, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
combined_df_v2 = pd.merge(combined_df_v2, records_v2, left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
combined_df_v2 = pd.merge(combined_df_v2, seeds, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
combined_df_v2 = pd.merge(combined_df_v2, seeds, left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
combined_df_v2 = combined_df_v2.rename(columns={'Seed': 'LSeed'})
combined_df_v2

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,TeamID_x,Wins_x,...,dr_y,ast_y,to_y,stl_y,blk_y,pf_y,Seed_x,TeamID_x,Seed_y,TeamID_y
0,2003,134,1421,92,1411,84,N,1,1421,13,...,24.800000,14.200000,15.233333,6.433333,2.233333,18.300000,16,1421,16,1411
1,2003,136,1112,80,1436,51,N,0,1112,25,...,25.724138,14.206897,14.068966,6.862069,2.965517,15.896552,1,1112,16,1436
2,2003,138,1112,96,1211,95,N,2,1112,25,...,25.322581,15.741935,14.548387,6.806452,3.516129,18.645161,1,1112,9,1211
3,2003,143,1112,88,1323,71,N,0,1112,25,...,26.870968,16.903226,12.774194,7.451613,5.645161,16.225806,1,1112,5,1323
4,2003,136,1113,84,1272,71,N,0,1113,18,...,25.965517,16.620690,13.793103,7.379310,5.068966,18.758621,10,1113,7,1272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2019,152,1438,63,1120,62,N,0,1438,29,...,21.941176,14.411765,12.147059,9.294118,4.764706,18.382353,1,1438,5,1120
1104,2019,154,1438,85,1403,77,N,1,1438,29,...,25.437500,14.031250,12.375000,7.375000,4.906250,17.781250,1,1438,3,1403
1105,2019,137,1439,66,1387,52,N,0,1439,24,...,25.942857,12.971429,12.714286,7.085714,4.057143,17.485714,4,1439,13,1387
1106,2019,139,1439,67,1251,58,N,0,1439,24,...,24.129032,14.129032,11.161290,6.161290,2.451613,15.612903,4,1439,12,1251


In [257]:
combined_df_v2.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'TeamID_x', 'Wins_x', 'Loses_x', 'ppg_x', 'fgm_x', 'fga_x',
       'fga3_x', 'ftm_x', 'fta_x', 'or_x', 'dr_x', 'ast_x', 'to_x', 'stl_x',
       'blk_x', 'pf_x', 'TeamID_y', 'Wins_y', 'Loses_y', 'ppg_y', 'fgm_y',
       'fga_y', 'fga3_y', 'ftm_y', 'fta_y', 'or_y', 'dr_y', 'ast_y', 'to_y',
       'stl_y', 'blk_y', 'pf_y', 'Seed_x', 'TeamID_x', 'Seed_y', 'TeamID_y'],
      dtype='object')

In [265]:
upsets_v2 = combined_df_v2[combined_df_v2['Seed_x'] > combined_df_v2['Seed_y']]
upsets_v2 = upsets_v2.rename(columns = {'LTeamID': 'HighSeedTeamID', 'WTeamID': 'LowSeedTeamID',
                                       'Wins_x': 'LowSeedSeasonWins', 'Loses_x': 'LowSeedSeasonLoses', 
                                        'ppg_x': 'LowSeedPPG', 'fgm_x': 'LowSeedFGM', 'fga_x': 'LowSeedFGA',
                                       'fga3_x': 'LowSeedFGA3', 'ftm_x': 'LowSeedFTM', 
                                        'fta_x': 'LowSeedFTA', 'or_x': 'LowSeedOR', 'dr_x': 'LowSeedDR', 
                                        'ast_x': 'LowSeedAST', 'to_x': 'LowSeedTO', 'stl_x': 'LowSeedSTL',
                                       'blk_x': 'LowSeedBLK', 'pf_x': 'LowSeedPF', 
                                        'Wins_y': 'HighSeedSeasonWins', 'Loses_y': 'HighSeedSeasonLoses', 
                                        'ppg_y': 'HighSeedPPG', 'fgm_y': 'HighSeedFGM',
                                       'fga_y': 'HighSeedFGA', 'fga3_y': 'HighSeedFGA3', 
                                        'ftm_y': 'HighSeedFTM' , 'fta_y': 'HighSeedFTA', 
                                        'or_y': 'HighSeedOR', 'dr_y': 'HighSeedDR', 'ast_y': 'HighSeedAST', 
                                        'to_y': 'HighSeedTO', 'stl_y': 'HighSeedSTL', 'blk_y': 'HighSeedBLK', 
                                        'pf_y': 'HighSeedPF', 'Seed_x': 'LowSeed', 'Seed_y': 'HighSeed'})
upsets_v2 = upsets_v2.drop(columns = ['WScore', 'LScore', 'WLoc', 'NumOT', 'TeamID_x', 'TeamID_y'])
upsets_v2['HighSeedWon'] = 0
upsets_v2

,Season,DayNum,LowSeedTeamID,HighSeedTeamID,LowSeedSeasonWins,LowSeedSeasonLoses,LowSeedPPG,LowSeedFGM,LowSeedFGA,LowSeedFGA3,...,HighSeedOR,HighSeedDR,HighSeedAST,HighSeedTO,HighSeedSTL,HighSeedBLK,HighSeedPF,LowSeed,HighSeed,HighSeedWon
4,2003,136,1113,1272,18,11,75.965517,27.206897,56.896552,12.586207,...,14.068966,25.965517,16.620690,13.793103,7.379310,5.068966,18.758621,10,7,0
5,2003,136,1141,1166,23,6,79.344828,26.620690,52.689655,17.931034,...,10.878788,23.181818,16.818182,13.363636,8.393939,4.454545,17.272727,11,6,0
8,2003,138,1163,1390,21,9,80.033333,29.533333,62.200000,15.700000,...,13.741935,25.129032,14.645161,14.322581,5.516129,3.354839,18.516129,5,4,0
11,2003,136,1211,1153,23,8,77.064516,26.064516,55.451613,19.064516,...,12.142857,23.392857,12.285714,10.607143,5.178571,4.250000,18.964286,9,8,0
16,2003,145,1242,1112,23,7,81.333333,30.233333,62.300000,14.133333,...,15.178571,27.642857,17.642857,14.785714,8.464286,4.214286,17.750000,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,2019,145,1403,1211,26,6,73.093750,26.156250,55.468750,19.718750,...,9.545455,29.090909,18.212121,10.363636,7.545455,5.484848,16.060606,3,1,0
1096,2019,152,1403,1277,26,6,73.093750,26.156250,55.468750,19.718750,...,10.823529,30.088235,18.941176,12.852941,5.235294,5.470588,16.911765,3,2,0
1097,2019,137,1414,1243,29,5,72.470588,26.941176,58.970588,18.882353,...,9.454545,23.818182,13.757576,11.272727,7.606061,2.272727,16.151515,13,4,0
1098,2019,137,1416,1433,23,8,72.129032,24.677419,53.290323,19.225806,...,10.843750,25.937500,13.875000,14.000000,7.968750,4.531250,19.812500,9,8,0


In [266]:
favorites_v2 = combined_df_v2[combined_df_v2['Seed_x'] <= combined_df_v2['Seed_y']]
favorites_v2 = favorites_v2.rename(columns = {'LTeamID': 'LowSeedTeamID', 'WTeamID': 'HighSeedTeamID',
                                       'Wins_y': 'LowSeedSeasonWins', 'Loses_y': 'LowSeedSeasonLoses', 
                                        'ppg_y': 'LowSeedPPG', 'fgm_y': 'LowSeedFGM', 'fga_y': 'LowSeedFGA',
                                       'fga3_y': 'LowSeedFGA3', 'ftm_y': 'LowSeedFTM', 
                                        'fta_y': 'LowSeedFTA', 'or_y': 'LowSeedOR', 'dr_y': 'LowSeedDR', 
                                        'ast_y': 'LowSeedAST', 'to_y': 'LowSeedTO', 'stl_y': 'LowSeedSTL',
                                       'blk_y': 'LowSeedBLK', 'pf_y': 'LowSeedPF', 
                                        'Wins_x': 'HighSeedSeasonWins', 'Loses_x': 'HighSeedSeasonLoses', 
                                        'ppg_x': 'HighSeedPPG', 'fgm_x': 'HighSeedFGM',
                                       'fga_x': 'HighSeedFGA', 'fga3_x': 'HighSeedFGA3', 
                                        'ftm_x': 'HighSeedFTM' , 'fta_x': 'HighSeedFTA', 
                                        'or_x': 'HighSeedOR', 'dr_x': 'HighSeedDR', 'ast_x': 'HighSeedAST', 
                                        'to_x': 'HighSeedTO', 'stl_x': 'HighSeedSTL', 'blk_x': 'HighSeedBLK', 
                                        'pf_x': 'HighSeedPF', 'Seed_y': 'LowSeed', 'Seed_x': 'HighSeed'})
favorites_v2 = favorites_v2.drop(columns = ['WScore', 'LScore', 'WLoc', 'NumOT', 'TeamID_x', 'TeamID_y'])
favorites_v2['HighSeedWon'] = 1
favorites_v2

,Season,DayNum,HighSeedTeamID,LowSeedTeamID,HighSeedSeasonWins,HighSeedSeasonLoses,HighSeedPPG,HighSeedFGM,HighSeedFGA,HighSeedFGA3,...,LowSeedOR,LowSeedDR,LowSeedAST,LowSeedTO,LowSeedSTL,LowSeedBLK,LowSeedPF,HighSeed,LowSeed,HighSeedWon
0,2003,134,1421,1411,13,16,71.206897,24.379310,56.793103,18.000000,...,13.166667,24.800000,14.200000,15.233333,6.433333,2.233333,18.300000,16,16,1
1,2003,136,1112,1436,25,3,85.214286,30.321429,65.714286,20.071429,...,12.965517,25.724138,14.206897,14.068966,6.862069,2.965517,15.896552,1,16,1
2,2003,138,1112,1211,25,3,85.214286,30.321429,65.714286,20.071429,...,11.935484,25.322581,15.741935,14.548387,6.806452,3.516129,18.645161,1,9,1
3,2003,143,1112,1323,25,3,85.214286,30.321429,65.714286,20.071429,...,11.387097,26.870968,16.903226,12.774194,7.451613,5.645161,16.225806,1,5,1
6,2003,136,1143,1301,21,8,74.482759,27.344828,58.724138,17.034483,...,9.733333,22.033333,14.666667,14.200000,7.766667,3.066667,18.666667,8,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,2019,145,1438,1345,29,3,71.843750,25.625000,53.593750,20.718750,...,12.218750,24.562500,14.437500,10.687500,6.500000,4.031250,17.500000,1,3,1
1103,2019,152,1438,1120,29,3,71.843750,25.625000,53.593750,20.718750,...,11.735294,21.941176,14.411765,12.147059,9.294118,4.764706,18.382353,1,5,1
1104,2019,154,1438,1403,29,3,71.843750,25.625000,53.593750,20.718750,...,8.843750,25.437500,14.031250,12.375000,7.375000,4.906250,17.781250,1,3,1
1105,2019,137,1439,1387,24,8,74.000000,25.750000,54.156250,24.343750,...,13.828571,25.942857,12.971429,12.714286,7.085714,4.057143,17.485714,4,13,1


In [302]:
# More detailed regular season stats, starting in 2003 season
train_detailed = pd.concat([favorites_v2, upsets_v2])

In [303]:
# Break DF into X and Y, and check shape to determine dimensions for neural network
X = train_detailed[train_detailed.columns[:-1]]
Y = train_detailed[train_detailed.columns[-1]]
X.shape

(1108, 36)

In [304]:
# use first 12 years for training, and last 5 for testing for about a 70/30 split
training_data_v2 = train_detailed[train_detailed['Season'] < 2015]
testing_data_v2 = train_detailed[train_detailed['Season'] >= 2015]

X_train_v2 = training_data_v2[training_data_v2.columns[:-1]]
Y_train_v2 = training_data_v2[training_data_v2.columns[-1]]
X_test_v2 = testing_data_v2[testing_data_v2.columns[:-1]]
Y_test_v2 = testing_data_v2[testing_data_v2.columns[-1]]

In [305]:
nn_v2 = define_model(X_train_v2)
nn_v2_128 = define_model(X_train_v2)
nn_v2_512 = define_model(X_train_v2)

In [306]:
history_v2 = nn_v2.fit(X_train_v2, Y_train_v2, epochs = 256, batch_size = 32)

Epoch 1/256
25/25 [==============================] - 1s 2ms/step - loss: 0.6637 - accuracy: 0.6381
Epoch 2/256
25/25 [==============================] - 0s 2ms/step - loss: 0.6108 - accuracy: 0.6816
Epoch 3/256
25/25 [==============================] - 0s 3ms/step - loss: 0.5582 - accuracy: 0.7083
Epoch 4/256
25/25 [==============================] - 0s 3ms/step - loss: 0.5360 - accuracy: 0.7129
Epoch 5/256
25/25 [==============================] - 0s 3ms/step - loss: 0.5368 - accuracy: 0.7075
Epoch 6/256
25/25 [==============================] - 0s 3ms/step - loss: 0.4923 - accuracy: 0.7387
Epoch 7/256
25/25 [==============================] - 0s 3ms/step - loss: 0.4629 - accuracy: 0.7661
Epoch 8/256
25/25 [==============================] - 0s 2ms/step - loss: 0.4601 - accuracy: 0.7658
Epoch 9/256
25/25 [==============================] - 0s 3ms/step - loss: 0.4415 - accuracy: 0.7812
Epoch 10/256
25/25 [==============================] - 0s 5ms/step - loss: 0.4085 - accuracy: 0.8050
Epoch 11/

25/25 [==============================] - 0s 6ms/step - loss: 0.0433 - accuracy: 0.9896
Epoch 84/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0454 - accuracy: 0.9842
Epoch 85/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0489 - accuracy: 0.9847
Epoch 86/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0387 - accuracy: 0.9872
Epoch 87/256
25/25 [==============================] - 0s 5ms/step - loss: 0.0351 - accuracy: 0.9847
Epoch 88/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0365 - accuracy: 0.9864
Epoch 89/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0248 - accuracy: 0.9936
Epoch 90/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0515 - accuracy: 0.9836
Epoch 91/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0493 - accuracy: 0.9789
Epoch 92/256
25/25 [==============================] - 0s 6ms/step - loss: 0.0427 - accuracy: 0.9885
Epoch 93/256


25/25 [==============================] - 0s 2ms/step - loss: 0.0115 - accuracy: 0.9985
Epoch 165/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9990
Epoch 166/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0245 - accuracy: 0.9949
Epoch 167/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0391 - accuracy: 0.9837
Epoch 168/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0522 - accuracy: 0.9788
Epoch 169/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0372 - accuracy: 0.9865
Epoch 170/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 0.9972
Epoch 171/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0355 - accuracy: 0.9882
Epoch 172/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0376 - accuracy: 0.9882
Epoch 173/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 0.9955
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0435 - accuracy: 0.9901
Epoch 246/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0361 - accuracy: 0.9920
Epoch 247/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 0.9963
Epoch 248/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0205 - accuracy: 0.9947
Epoch 249/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0174 - accuracy: 0.9944
Epoch 250/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 0.9977
Epoch 251/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0256 - accuracy: 0.9955
Epoch 252/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0368 - accuracy: 0.9913
Epoch 253/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0175 - accuracy: 0.9974
Epoch 254/256
25/25 [==============================] - 0s 2ms/step - loss: 0.0375 - accuracy: 0.9857
Epoc

In [307]:
history_v2_128 = nn_v2_128.fit(X_train_v2, Y_train_v2, epochs = 128, batch_size = 32)

Epoch 1/128
25/25 [==============================] - 1s 3ms/step - loss: 0.6571 - accuracy: 0.5990
Epoch 2/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5627 - accuracy: 0.7274
Epoch 3/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5597 - accuracy: 0.7096
Epoch 4/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.7272
Epoch 5/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5146 - accuracy: 0.7222
Epoch 6/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5090 - accuracy: 0.7369
Epoch 7/128
25/25 [==============================] - 0s 2ms/step - loss: 0.5089 - accuracy: 0.7344
Epoch 8/128
25/25 [==============================] - 0s 3ms/step - loss: 0.4872 - accuracy: 0.7418
Epoch 9/128
25/25 [==============================] - 0s 2ms/step - loss: 0.4246 - accuracy: 0.7968
Epoch 10/128
25/25 [==============================] - 0s 3ms/step - loss: 0.4148 - accuracy: 0.8112
Epoch 11/

25/25 [==============================] - 0s 2ms/step - loss: 0.0320 - accuracy: 0.9880
Epoch 84/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0566 - accuracy: 0.9817
Epoch 85/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0318 - accuracy: 0.9906
Epoch 86/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0434 - accuracy: 0.9891
Epoch 87/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0488 - accuracy: 0.9778
Epoch 88/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0426 - accuracy: 0.9859
Epoch 89/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0399 - accuracy: 0.9880
Epoch 90/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0200 - accuracy: 0.9949
Epoch 91/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 0.9950
Epoch 92/128
25/25 [==============================] - 0s 2ms/step - loss: 0.0373 - accuracy: 0.9839
Epoch 93/128


In [308]:
history_v2_512 = nn_v2_512.fit(X_train_v2, Y_train_v2, epochs = 512, batch_size = 32)

Epoch 1/512
25/25 [==============================] - 1s 3ms/step - loss: 0.6430 - accuracy: 0.6620
Epoch 2/512
25/25 [==============================] - 0s 2ms/step - loss: 0.5878 - accuracy: 0.7014
Epoch 3/512
25/25 [==============================] - 0s 2ms/step - loss: 0.5236 - accuracy: 0.7502
Epoch 4/512
25/25 [==============================] - 0s 3ms/step - loss: 0.5483 - accuracy: 0.7131
Epoch 5/512
25/25 [==============================] - 0s 3ms/step - loss: 0.4815 - accuracy: 0.7513
Epoch 6/512
25/25 [==============================] - 0s 3ms/step - loss: 0.4787 - accuracy: 0.7509
Epoch 7/512
25/25 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.7690
Epoch 8/512
25/25 [==============================] - 0s 2ms/step - loss: 0.4230 - accuracy: 0.7926
Epoch 9/512
25/25 [==============================] - 0s 2ms/step - loss: 0.4189 - accuracy: 0.8080
Epoch 10/512
25/25 [==============================] - 0s 2ms/step - loss: 0.3769 - accuracy: 0.8329
Epoch 11/

25/25 [==============================] - 0s 2ms/step - loss: 0.0680 - accuracy: 0.9821
Epoch 84/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0367 - accuracy: 0.9902
Epoch 85/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0449 - accuracy: 0.9827
Epoch 86/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 0.9894
Epoch 87/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9985
Epoch 88/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0494 - accuracy: 0.9822
Epoch 89/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0318 - accuracy: 0.9833
Epoch 90/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0297 - accuracy: 0.9921
Epoch 91/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0300 - accuracy: 0.9907
Epoch 92/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0343 - accuracy: 0.9929
Epoch 93/512


25/25 [==============================] - 0s 3ms/step - loss: 0.0172 - accuracy: 0.9924
Epoch 165/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.9967
Epoch 166/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 0.9892
Epoch 167/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0221 - accuracy: 0.9924
Epoch 168/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0262 - accuracy: 0.9886
Epoch 169/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0419 - accuracy: 0.9876
Epoch 170/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0302 - accuracy: 0.9939
Epoch 171/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0243 - accuracy: 0.9915
Epoch 172/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 0.9966
Epoch 173/512
25/25 [==============================] - 0s 4ms/step - loss: 0.0141 - accuracy: 0.9955
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0273 - accuracy: 0.9931
Epoch 246/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0132 - accuracy: 0.9966
Epoch 247/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 0.9948
Epoch 248/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 0.9916
Epoch 249/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0281 - accuracy: 0.9909
Epoch 250/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0228 - accuracy: 0.9966
Epoch 251/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 0.9925
Epoch 252/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0296 - accuracy: 0.9928
Epoch 253/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0096 - accuracy: 0.9982
Epoch 254/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0135 - accuracy: 0.9961
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0102 - accuracy: 0.9944
Epoch 327/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 0.9989
Epoch 328/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 0.9977
Epoch 329/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0067 - accuracy: 0.9986
Epoch 330/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0172 - accuracy: 0.9953
Epoch 331/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0321 - accuracy: 0.9916
Epoch 332/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0367 - accuracy: 0.9869
Epoch 333/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0213 - accuracy: 0.9930
Epoch 334/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 0.9942
Epoch 335/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0359 - accuracy: 0.9936
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 0.9959
Epoch 407/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0135 - accuracy: 0.9962
Epoch 408/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 409/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0070 - accuracy: 0.9989
Epoch 410/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 0.9985
Epoch 411/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0525 - accuracy: 0.9857
Epoch 412/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 0.9928
Epoch 413/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0151 - accuracy: 0.9918
Epoch 414/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0196 - accuracy: 0.9943
Epoch 415/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0096 - accuracy: 0.9951
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 0.9992
Epoch 488/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0241 - accuracy: 0.9971
Epoch 489/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0100 - accuracy: 0.9992
Epoch 490/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 491/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 0.9987
Epoch 492/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 0.9980
Epoch 493/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0083 - accuracy: 0.9956
Epoch 494/512
25/25 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 0.9985
Epoch 495/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0086 - accuracy: 0.9996
Epoch 496/512
25/25 [==============================] - 0s 2ms/step - loss: 0.0228 - accuracy: 0.9914
Epoc

In [309]:
plot_model_accuracy_and_loss(history_v2, 'model_accuracy_v2_256.png', 'model_loss_v2_256.png')
plot_model_accuracy_and_loss(history_v2_128, 'model_accuracy_v2_128.png', 'model_loss_v2_128.png')
plot_model_accuracy_and_loss(history_v2_512, 'model_accuracy_v2_512.png', 'model_loss_v2_512.png')

<Figure size 432x288 with 0 Axes>

In [299]:
# Will use 256 epochs going forward. 
predictions = (nn_v2.predict(X_test_v2, batch_size = 32) > 0.5).astype("int32")
mean_squared_error(Y_test_v2, predictions)

0.3212121212121212

In [300]:
accuracy_score(Y_test_v2, predictions)

0.6787878787878788

In [301]:
confusion_matrix(Y_test_v2, predictions)

array([[ 30,  56],
       [ 50, 194]], dtype=int64)